# 4. Constructing AR-GARCH-X model

This template is used to calculate and show plots of the AR-GARCH-X model, which makes use of the ArGarchX class. This class has several methods in order to compute, with the help of the Maximum Likelihood method the estimators that maximize the Quasi Log Likelihood. Via this template several models will be constructed and tested, in order to check the effect of public sentiment on volatility, and it will be checked whether adding these variables increases predictive accuracy.

## 4.1. Load packages and data

Here, load the packages, data and colors for the main analysis and for the construction of plots, also define export locations

### 4.1.1. Load packages

In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import os, sys
import itertools
from datetime import datetime
from scipy.stats import chi2
from scipy.stats import norm

# Load data that returns tweets
sys.path.insert(0, os.path.abspath(r'C:\Users\Jonas\PycharmProjects\TwitterSentimentGARCH2021\Code\GARCH model\tvGARCH models'))
from tv_garch_models import tvArmaApARCHX, tvArmaXapARCH, tvArmaApArchXGarch
from tv_garch_estimation import QuasiMaximumLikelihoodEstimator

# Surpress warnings
import warnings
warnings.filterwarnings("ignore")

### 4.1.2. Colors for plots

In [2]:
colors = ['seagreen', 'mediumaquamarine', 'steelblue', 'cornflowerblue', 'navy', 'black']

### 4.1.3. Load data

Load company name data and DataFrame per company containing all the sentiment, return and control variable data

In [3]:
# Specify location of data + file name and location of storage
company_loc = r'C:\Users\Jonas\Documents\Data'
file_name_comp = '\company_ticker_list_all.xlsx'

# Access company names DataFrame
df_comp_names = pd.read_excel(company_loc + file_name_comp)

Specify location where all company specific data is stored

In [4]:
# Specify name and location
data_loc = r'C:\Users\Jonas\Documents\Data\Total_data'

Specify location where to store results

In [5]:
# Specify location where daily sentiment scores must be stored
store_loc = r'C:\Users\Jonas\Documents\Data\Results'

## 4.2. Analysis

This section performs the main analysis and will calculate the results for each company in the selection.

### 4.2.1. Create models

First, create the different columns in the total dataset that need to be evaluated and serve as input into the models

In [6]:
# Define possible exogenous columns
control_cols = ['VIX', 'TEDRATE']
sent_cols = ['sentiment', 'n_tweets', 'n_interactions']

# Define data on which to impose GARCH structure, and provide column names
x_garch_cols = [f'sigma2_{col}' for col in sent_cols]

### 4.2.2. Calculate optimal number of lags

Do a grid search for various values of h, in the second model, where the sentiment variables enter the model in the conditional volatility equation. The value for h that maximizes the QLL is used throughout the research.

In [8]:
# Find lags specification for each model
df_model_specification = pd.read_csv(data_loc + f'\\lags\\model_params.csv')

# For every company in the dataset, read the data and construct the appropriate model
for company in df_comp_names['Company']:
    
    # Read data
    data_name = f'\\total data {company}.csv' 
    
    # Get df_total
    df_total = pd.read_csv(data_loc + data_name)
       
    # Drop NaNs
    df_total = df_total.fillna(0)
    
    # Unpack the optimal lags of the ARMA procedure from df_model_specification
    lags_per_model = [literal_eval(x) for x in df_model_specification[company]]
    lags_arma = list(lags_per_model[0])
    
    # Calculate optimal parameters for each model
    h_vals = [10, 15, 25, 50, 100]
    dict_h = {}
    
    for h in h_vals:   
        qmle = QuasiMaximumLikelihoodEstimator(df_total, 'returns', control_cols, h, lags_arma,
                                               sent_cols, list(lags_per_model[2]), params=None, 
                                               model_type='asym')
    
        minimization_result, psi_hat, likelihood, df_params = qmle.optimize_likelihood()
        
        dict_h[h] = likelihood
    
        print(f"Likelihood with the smooth transition operator h = {h} is {likelihood}")
        
    print(f"Among the suggested values for h, the best value is {max(dict_h, key=dict_h.get)}, with likelihood {max(dict_h)}")

Likelihood with the smooth transition operator h = 10 is -1.9486271886993813
Likelihood with the smooth transition operator h = 15 is -1.945442916947736
Likelihood with the smooth transition operator h = 25 is -1.9457677823207047
Likelihood with the smooth transition operator h = 50 is -1.9448963750649415
Likelihood with the smooth transition operator h = 100 is -1.9448924266510739
Among the suggested values for h, the best value is 100, with likelihood 100
Likelihood with the smooth transition operator h = 10 is -2.2142867220679365
Likelihood with the smooth transition operator h = 15 is -2.2136188830748833
Likelihood with the smooth transition operator h = 25 is -2.2129875986274072
Likelihood with the smooth transition operator h = 50 is -2.2170652166586855
Likelihood with the smooth transition operator h = 100 is -2.2169744038915837
Among the suggested values for h, the best value is 25, with likelihood 100
Likelihood with the smooth transition operator h = 10 is -1.602168927746788


Likelihood with the smooth transition operator h = 25 is -2.3356617764162237
Likelihood with the smooth transition operator h = 50 is -2.335805803051859
Likelihood with the smooth transition operator h = 100 is -2.3358096475272343
Among the suggested values for h, the best value is 10, with likelihood 100
Likelihood with the smooth transition operator h = 10 is -3.3554292120486244
tolerance level is changed to 1e-06
     fun: 3.360083893148399
     jac: array([-4.93231353e-05, -1.69740006e-04, -2.41364294e-04,  5.17437158e-04,
        4.62912538e-05,  1.00019976e-02, -6.18667413e-04,  8.49760164e-02,
       -6.21259180e-07,  4.33937221e-02, -1.95067300e-04, -2.53515904e-05,
       -3.05844369e-04,  1.01929660e-04, -7.67816283e-06, -8.42864981e-06,
        3.37924490e-06, -1.33494709e-04,  1.74767458e-05, -1.70873845e-04,
       -8.04861683e-05,  1.18860028e-02,  7.24611762e-02,  5.36260744e-02,
        8.20429118e-05, -1.54785165e-04, -1.25908803e-03, -1.43251487e-04])
 message: 'Optim

### 4.2.3. Calculate optimal time-varying parameters and standard errors

In this section, for each model, the optimal parameters are calculated and the standard error of these parameters. 

In [ ]:
h = max(dict_h, key=dict_h.get)

In [ ]:
# Find lags specification for each model
df_model_specification = pd.read_csv(data_loc + f'\\lags\\model_params.csv')

# For every company in the dataset, read the data and construct the appropriate model
for company in df_comp_names['Company']:
    
    # Read data
    data_name = f'\\total data {company}.csv' 
    
    # Get df_total
    df_total = pd.read_csv(data_loc + data_name)
       
    # Drop NaNs
    df_total = df_total.fillna(0)
    
    # Unpack the optimal lags of the ARMA procedure from df_model_specification
    lags_per_model = [literal_eval(x) for x in df_model_specification[company]]
    lags_arma = list(lags_per_model[0])
    
    # Calculate optimal parameters for each model
    
    # - Benchmark model
    #qmle = QuasiMaximumLikelihoodEstimator(df_total, 'returns', control_cols, lags_arma)
    
    #minimization_result, psi_hat, likelihood, df_params = qmle.optimize_likelihood()
    
    #vcov = qmle.vcov(psi_hat)
    #df_params['errors'] = np.sqrt(np.diag(vcov))
    #df_params['t-stat'] = psi_hat / np.sqrt(np.diag(vcov))
    
    
    # - Model ARMA-X-apARCH
    qmle1 = QuasiMaximumLikelihoodEstimator(df_total, 'returns', control_cols, h, lags_arma,
                                            sent_cols, list(lags_per_model[1]), params=None, 
                                            model_type='mean-x')
    
    minimization_result1, psi_hat1, likelihood1, df_params1 = qmle1.optimize_likelihood()
    
    vcov1 = qmle1.vcov(psi_hat1)
    df_params1['errors'] = np.sqrt(np.diag(vcov1))
    df_params1['t-stat'] = psi_hat1 / np.sqrt(np.diag(vcov1))
    
    # - Model ARMA-apARCH-X
    qmle2 = QuasiMaximumLikelihoodEstimator(df_total, 'returns', control_cols, h, lags_arma,
                                            sent_cols, list(lags_per_model[2]), params=None, 
                                            model_type='asym')
    
    minimization_result2, psi_hat2, likelihood2, df_params2 = qmle2.optimize_likelihood()
    
    vcov2 = qmle2.vcov(psi_hat2)
    df_params2['errors'] = np.sqrt(np.diag(vcov2))
    df_params2['t-stat'] = psi_hat2  /  np.sqrt(np.diag(vcov2))
    
    # - Model ARMA-apARCH-(X-ARMA-GARCH)
    exog_lags = list(lags_per_model[3])[:len(sent_cols)]
    x_garch_lags = list(lags_per_model[3])[len(sent_cols):]
    qmle3 = QuasiMaximumLikelihoodEstimator(df_total, 'returns', control_cols, h, lags_arma,
                                            sent_cols, exog_lags, params=None,
                                            model_type='x-garch', x_garch_cols=x_garch_cols, 
                                            x_garch_lags=x_garch_lags)
    
    minimization_result3, psi_hat3, likelihood3, df_params3 = qmle3.optimize_likelihood()
    
    vcov3 = qmle3.vcov(psi_hat3)
    df_params3['errors'] = np.sqrt(np.diag(vcov3))
    df_params3['t-stat'] = psi_hat3  / np.sqrt(np.diag(vcov3))
    
    # Store all parameters into a .csv file
    #df_params.to_csv(data_loc + f'\\parameters\\benchmark\\tv params {company}.csv', index=False)
    df_params1.to_csv(data_loc + f'\\parameters\\model1\\tv params {company}.csv', index=False)
    df_params2.to_csv(data_loc + f'\\parameters\\model2\\tv params {company}.csv', index=False)
    df_params3.to_csv(data_loc + f'\\parameters\\model3\\tv params {company}.csv', index=False)

Define a Likelihood calculation functionality that can be used to quickly calculate the likelihood given residuals and conditional volatility

In [ ]:
def quasi_log_likelihood(sigma2, et):
    # QMLE from Franq and Thieu
    lls = np.log(sigma2) + ((et ** 2) / sigma2) + np.log(2 * np.pi)

    # Calculate Quasi Maximum Likelihood
    ll = np.nan_to_num(lls).sum()

    return -ll

## 4.3. Summary of results

Here, calculate and plot the constructed values of the conditional volatility model, conditional mean model and the distribution of the innovations $z_t$.

In [ ]:
# Find lags specification for each model
df_model_specification = pd.read_csv(data_loc + f'\\lags\\model_params.csv')

# For every company in the dataset, read the data and construct the appropriate model
for company in df_comp_names['Company']:    
    # Read data
    data_name = f'\\total data {company}.csv' 
    
    # Get df_total
    df_total = pd.read_csv(data_loc + data_name)
       
    # Drop NaNs
    df_total = df_total.fillna(0)
    
    # Unpack the optimal lags of the ARMA procedure from df_model_specification
    lags_per_model = [literal_eval(x) for x in df_model_specification[company]]
    lags_arma = list(lags_per_model[0])
    
    # Open parameter files
    df_params = pd.read_csv(data_loc + f'\\parameters\\benchmark\\tv params {company}.csv')
    df_params1 = pd.read_csv(data_loc + f'\\parameters\\model1\\tv params {company}.csv')
    df_params2 = pd.read_csv(data_loc + f'\\parameters\\model2\\tv params {company}.csv')
    df_params3 = pd.read_csv(data_loc + f'\\parameters\\model3\\tv params {company}.csv')
    
    # Calculate sigma2 values for each model
    benchmark = tvArmaApARCHX(df_total, 'returns', control_cols, h, lags_arma, params=df_params['psi_hat'].tolist())
    
    
    model1 = tvArmaXapARCH(df_total, 'returns', control_cols, h, lags_arma, sent_cols, list(lags_per_model[1]), 
                         params = df_params1['psi_hat'].tolist())
    
    model2 = tvArmaApARCHX(df_total, 'returns', control_cols, h, lags_arma, sent_cols, list(lags_per_model[2]), 
                         params = df_params2['psi_hat'].tolist())
    
    exog_lags = list(lags_per_model[3])[:len(sent_cols)]
    x_garch_lags = list(lags_per_model[3])[len(sent_cols):]
    
    model3 = tvArmaApArchXGarch(df_total, 'returns', control_cols, h, lags_arma, sent_cols, exog_lags, 
                              params = df_params3['psi_hat'].tolist(),
                              xgarch_cols=x_garch_cols, lag_exog_sigma=x_garch_lags)
    
    sigma2, et = benchmark.conditional_volatility()
    sigma2_1, et_1 = model1.conditional_volatility()
    sigma2_2, et_2 = model2.conditional_volatility()
    sigma2_3, et_3 = model3.conditional_volatility()
    
    vars_, ets = [sigma2_1, sigma2_2, sigma2_3], [et_1, et_2, et_3]
    
    # Calculate likelihood for all models
    ll_benchmark = quasi_log_likelihood(sigma2, et)
    ll_1, ll_2, ll_3 = quasi_log_likelihood(sigma2_1, et_1), quasi_log_likelihood(sigma2_2, et_2), quasi_log_likelihood(sigma2_3, et_3)
    likelihoods = [ll_1, ll_2, ll_3]
    
    # Now, construct plots for all models
    fig, axs = plt.subplots(figsize = (20,4), nrows = 1, ncols = 3)
    
    first_date, last_date = df_total.date.iloc[0], df_total.date.iloc[-1]
    n = 150  # keeps every 150th label (around half a year)

    for j in range(len(axs)):
        axs[j].plot(np.sqrt(ets[j] ** 2), c='black', linestyle='-.', alpha=0.7)
        axs[j].plot(np.sqrt(sigma2), c='yellow', linestyle='--', alpha=0.85)
        axs[j].plot(np.sqrt(vars_[j]), c=colors[j], alpha=0.7)
        
        # Set title and xticklabels
        axs[j].set_title(f'Conditional volatility of: {company}' + '\n' f'LR {-2*(ll_benchmark - likelihoods[j])}' + '\n' + f'Period: {first_date} to {last_date}')
    
    plt.tight_layout()
    
    # Plot KDE plot of innovations
    z_ts = [et_1 /np.sqrt(sigma2_1), et_2 / np.sqrt(sigma2_2), et_3 / np.sqrt(sigma2_3)]

    fig2, axs = plt.subplots(figsize = (20,4), nrows = 1, ncols = len(z_ts))
    for i, ax in enumerate(axs):
        sns.kdeplot(z_ts[i], ax=ax, color=colors[i])
        ax2 = ax.twinx()
        ax2.hist(z_ts[i], color=colors[i+2], bins=25, rwidth=0.9)

    plt.tight_layout()
    
    # Store figures as PNG
    fig.savefig(store_loc + f'\\plots\\tv plot conditional vol {company}')
    fig2.savefig(store_loc + f'\\plots\\tv kde plot of innovations of {company}')

## 4.5 Export parameter DataFrames

This section creates and exports the DataFrames with parameter estimates of each model included in this research for all companies in this research.

In [ ]:
# Create empty DataFrames to store results in
df_psi_hat_1 = pd.DataFrame()
df_psi_hat_2 = pd.DataFrame()
df_psi_hat_3 = pd.DataFrame()

for i in range(len(df_comp_names['Company'])):
    company = df_comp_names['Company'].iloc[i]

    # Store all parameters into a .csv file
    df_params1 = pd.read_csv(data_loc + f'\\parameters\\model1\\tv params {company}.csv')
    df_params2 = pd.read_csv(data_loc + f'\\parameters\\model2\\tv params {company}.csv')
    df_params3 = pd.read_csv(data_loc + f'\\parameters\\model3\\tv params {company}.csv')
    
    # Compile errors in parentheses
    errors1 = df_params1['errors'].apply(lambda x: '%.1e' % x).apply(lambda x: f' ({x})')
    
    # Get p-value based on t-statistic
    p = df_params1['t-stat'].apply(lambda x: ''.join(['*' for alpha in [0.1, 0.05, 0.01] if abs(x) >= norm.ppf(1-alpha)]))    
    df_params = pd.DataFrame(columns = ['param names', f'psi_hat_{company}'])
    
    # Create DataFrame with errors in brackets under estimates, starts indicating the significance
    for j in range(len(df_params1)):
        df_params.loc[len(df_params)] = [df_params1.iloc[j]['param names'], 
                                         df_params1.iloc[j]['psi_hat'].round(3).astype(str) + p[j]]
        df_params.loc[len(df_params)] = ['error_' + df_params1.iloc[j]['param names'], errors1[j]]
            
    df_params1 = df_params
    
    # Compile errors in parentheses
    errors2 = df_params2['errors'].apply(lambda x: '%.1e' % x).apply(lambda x: f' ({x})')
    
    # Get p-value based on t-statistic
    p = df_params2['t-stat'].apply(lambda x: ''.join(['*' for alpha in [0.1, 0.05, 0.01] if abs(x) >= norm.ppf(1-alpha)]))
    df_params = pd.DataFrame(columns = ['param names', f'psi_hat_{company}'])
    
    # Create DataFrame with errors in brackets under estimates, starts indicating the significance
    for j in range(len(df_params2)):
        df_params.loc[len(df_params)] = [df_params2.iloc[j]['param names'], 
                                         df_params2.iloc[j]['psi_hat'].round(3).astype(str) + p[j]]
        df_params.loc[len(df_params)] = ['error_' + df_params2.iloc[j]['param names'], errors2[j]]
            
    df_params2 = df_params
   
    # Compile errors in parentheses
    errors3 = df_params3['errors'].apply(lambda x: '%.1e' % x).apply(lambda x:  f' ({x})')
    
    # Get p-value based on t-statistic
    p = df_params3['t-stat'].apply(lambda x: ''.join(['*' for alpha in [0.1, 0.05, 0.01] if abs(x) >= norm.ppf(1-alpha)])) 
    df_params = pd.DataFrame(columns = ['param names', f'psi_hat_{company}'])
    
    # Create DataFrame with errors in brackets under estimates, starts indicating the significance
    for j in range(len(df_params3)):
        df_params.loc[len(df_params)] = [df_params3.iloc[j]['param names'], 
                                         df_params3.iloc[j]['psi_hat'].round(3).astype(str) + p[j]]
        df_params.loc[len(df_params)] = ['error_' + df_params3.iloc[j]['param names'], errors3[j]]
            
    df_params3 = df_params
    
    if i == 0:        
        df_psi_hat_1 = df_params1
        df_psi_hat_2 = df_params2
        df_psi_hat_3 = df_params3
    else:
        df_psi_hat_1 = df_psi_hat_1.merge(df_params1[['param names', f'psi_hat_{company}']], 
                                          on='param names', 
                                          how='outer') 

        df_psi_hat_2 = df_psi_hat_2.merge(df_params2[['param names', f'psi_hat_{company}']], 
                                          on='param names', 
                                          how='outer')
    

        df_psi_hat_3 = df_psi_hat_3.merge(df_params3[['param names', f'psi_hat_{company}']], 
                                          on='param names', 
                                          how='outer')
        
    # Save DataFrames to .csv
    df_psi_hat_1.to_csv(data_loc + f'\\parameters\\tv params model1.csv')
    df_psi_hat_2.to_csv(data_loc + f'\\parameters\\tv params model2.csv')
    df_psi_hat_3.to_csv(data_loc + f'\\parameters\\tv params model3.csv')

---------
---------